In [13]:
## Import the data stored in dynamo and 'unmarshal' it
import json
import sys
from boto3.dynamodb.types import TypeDeserializer

with open('training_data.json') as f:
    marshaled_data = f.readlines()

# Dynamo has a weird format that we need to convert to JSON
type_deserializer = TypeDeserializer()
    
# Write the deserialized data to the output file
with open("deserialized_data.json", 'w') as file:
    newlines = [] #The file is composed of JSON objects separated by newlines, not directly usable
    for line in marshaled_data:
        decoded_line = json.loads(line)['Item']
        new_line = {k:int(type_deserializer.deserialize(v)) if type(k) is not str else type_deserializer.deserialize(v) for k,v in decoded_line.items()}
        newlines.append(str(new_line).replace("Decimal('","").replace("')","").replace("'",'"'))
    file.write("["+",\n".join(newlines)+"]")
    unmarshaled_data = json.loads("["+",\n".join(newlines)+"]")

In [14]:
## Convert the unmarshaled data to use relative coordinates (the units are pixels)
samples = []
    
for item in unmarshaled_data:
    translation = item['translation']
    timeframe = item['end_timestamp'] - item['start_timestamp']
    path = item['raw_path']
    
    steps = []
    prev_point = None
    for point in path:
        if prev_point is not None:
            dx = point['x'] - prev_point['x']
            dy = point['y'] - prev_point['y']
            dt = point['t'] - prev_point['t']
            if dt < 3:
                dt = 3 #near 0 values are unacceptable but I don't feel like looking into it further
            steps.append({"x": dx, "y": dy, "t": dt})
        prev_point = point
        
    # Construct the new object
    samples.append({"translation": {"x": translation[0], "y": translation[1], "t": timeframe}, "steps": steps})

In [12]:
## I so far have not come up with a good model but my attempts are labeled main*.py